In [1]:
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical


In [2]:
# Load pre-trained ResNet50 model + higher-level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add global average pooling layer and a dense output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 [==============================] - 62s 1us/step


In [3]:
def load_images_and_labels(image_paths, labels):
    images = []
    valid_labels = []
    
    for image_path, label in zip(image_paths, labels):
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Warning: Unable to load image at {image_path}. Skipping...")
            continue
        
        # Detect the ROI using the genetic algorithm
        detected_rash = genetic_algorithm(image)
        
        # Extract ROI and resize to 224x224 for ResNet
        x, y, w, h = detected_rash
        roi = image[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (224, 224))
        
        # Preprocess the image for ResNet
        roi_resized = preprocess_input(roi_resized)
        
        images.append(roi_resized)
        valid_labels.append(label)
    
    return np.array(images), np.array(valid_labels)


In [4]:
# Load and preprocess the dataset
X_train, y_train = load_images_and_labels(train_image_paths, train_labels)
X_val, y_val = load_images_and_labels(val_image_paths, val_labels)

# One-hot encode labels
y_train = to_categorical(le.fit_transform(y_train), num_classes=2)
y_val = to_categorical(le.transform(y_val), num_classes=2)

# Data augmentation (optional)
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

# Train the model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=10,
    verbose=2
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=2)
print(f"Validation Accuracy: {val_accuracy:.2f}")


NameError: name 'train_image_paths' is not defined

In [ ]:
# Unfreeze the last few layers of ResNet
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training (fine-tuning)
history_fine = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=10,
    verbose=2
)


In [6]:
import os
import cv2
import numpy as np
import random
from skimage.feature import local_binary_pattern
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/Train_rash/Train_2_Cases"
val_path = "/Users/niteshyadav/Lyme_Disease/Validation_rash/Validation_2_Cases"

def load_image_paths_and_labels(data_dir):
    image_paths = []
    labels = []
    for label in ['Positive', 'Negative']:
        label_dir = os.path.join(data_dir, label)
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)
    return image_paths, labels

# Load paths and labels for training and validation data
train_image_paths, train_labels = load_image_paths_and_labels(train_path)
val_image_paths, val_labels = load_image_paths_and_labels(val_path)

def initialize_population(image, pop_size=10):
    population = []
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    _, thresh = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = mask1 + mask2
    
    combined_mask = cv2.bitwise_and(thresh, red_mask)
    
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours[:pop_size]:
        x, y, w, h = cv2.boundingRect(contour)
        population.append((x, y, w, h))
    
    h, w = gray_image.shape
    while len(population) < pop_size:
        rand_x = random.randint(0, w - 50)
        rand_y = random.randint(0, h - 50)
        width = random.randint(10, 50)
        height = random.randint(10, 50)
        population.append((rand_x, rand_y, width, height))
    
    return population

def fitness_function(candidate, image):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(hsv_roi, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_roi, lower_red2, upper_red2)
    red_mask = mask1 + mask2
    red_area = np.sum(red_mask) / (w * h)
    
    aspect_ratio = float(w) / h
    shape_score = 1.0 if 0.75 <= aspect_ratio <= 1.25 else 0.5
    
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    texture_score = np.mean(hist)
    
    fitness = red_area * 0.4 + shape_score * 0.3 + texture_score * 0.3
    return fitness

def crossover(parent1, parent2):
    x1, y1, w1, h1 = parent1
    x2, y2, w2, h2 = parent2
    child1 = (x1, y2, w1, h2)
    child2 = (x2, y1, w2, h1)
    return child1, child2

def mutate(candidate, image_shape, mutation_rate=0.1):
    if random.random() < mutation_rate:
        h, w = image_shape[:2]
        x, y, width, height = candidate
        x = random.randint(0, w - 50)
        y = random.randint(0, h - 50)
        width = random.randint(10, 50)
        height = random.randint(10, 50)
        candidate = (x, y, width, height)
    return candidate

def select(population, fitnesses, num_to_select):
    selected = list(zip(population, fitnesses))
    selected.sort(key=lambda x: x[1], reverse=True)
    return [candidate for candidate, fitness in selected[:num_to_select]]

def genetic_algorithm(image, num_generations=20, pop_size=10, mutation_rate=0.1):
    population = initialize_population(image, pop_size)
    for generation in range(num_generations):
        fitnesses = [fitness_function(candidate, image) for candidate in population]
        population = select(population, fitnesses, pop_size // 2)
        new_population = []
        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, image.shape, mutation_rate))
            new_population.append(mutate(child2, image.shape, mutation_rate))
        population = new_population
    best_candidate = max(population, key=lambda candidate: fitness_function(candidate, image))
    return best_candidate

# Load pre-trained ResNet50 model + custom layers for transfer learning
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model using the legacy Adam optimizer
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

def load_images_and_labels(image_paths, labels):
    data = []
    valid_labels = []
    
    for image_path, label in zip(image_paths, labels):
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Warning: Unable to load image at {image_path}. Skipping...")
            continue
        
        detected_rash = genetic_algorithm(image)
        roi = image[detected_rash[1]:detected_rash[1] + detected_rash[3], detected_rash[0]:detected_rash[0] + detected_rash[2]]
        roi_resized = cv2.resize(roi, (150, 150))
        data.append(roi_resized)
        valid_labels.append(label)
    
    return np.array(data), np.array(valid_labels)

# Load and preprocess the dataset
X_train, y_train = load_images_and_labels(train_image_paths, train_labels)
X_val, y_val = load_images_and_labels(val_image_paths, val_labels)

# Normalize the images
X_train = X_train / 255.0
X_val = X_val / 255.0

# Encode labels and one-hot encode them
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

# Create data generators for augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator()

# Training the model
batch_size = 16
epochs = 10

model.fit(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size),
    epochs=epochs,
    verbose=1
)

# Evaluate the model on the validation set
y_val_pred = model.predict(X_val)
y_val_pred = np.argmax(y_val_pred, axis=1)
y_val_true = np.argmax(y_val, axis=1)
accuracy = accuracy_score(y_val_true, y_val_pred)
print(f"Validation Accuracy: {accuracy:.2f}")


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'ICC Profile': 0h: PCS illuminant is not D50


Epoch 1/10
23/23 [==============================] - 21s 822ms/step - loss: 1.6930 - accuracy: 0.5309 - val_loss: 621.5412 - val_accuracy: 0.4138
Epoch 2/10
23/23 [==============================] - 19s 816ms/step - loss: 0.7644 - accuracy: 0.5927 - val_loss: 2175.8862 - val_accuracy: 0.4138
Epoch 3/10
23/23 [==============================] - 19s 820ms/step - loss: 0.7162 - accuracy: 0.5730 - val_loss: 2.6880 - val_accuracy: 0.5862
Epoch 4/10
23/23 [==============================] - 19s 823ms/step - loss: 0.7151 - accuracy: 0.5758 - val_loss: 0.6923 - val_accuracy: 0.5977
Epoch 5/10
23/23 [==============================] - 19s 814ms/step - loss: 0.6940 - accuracy: 0.5758 - val_loss: 0.6906 - val_accuracy: 0.5862
Epoch 6/10
23/23 [==============================] - 19s 831ms/step - loss: 0.6706 - accuracy: 0.6236 - val_loss: 0.6919 - val_accuracy: 0.5862
Epoch 7/10
23/23 [==============================] - 19s 815ms/step - loss: 0.6704 - accuracy: 0.6124 - val_loss: 0.6917 - val_accuracy: 0